In [ ]:
!python --version

In [ ]:
import tensorflow as tf
#import tensorflow_datasets as tfds
print("TensorFlow version:", tf.__version__)
#print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

In [ ]:
# import du dataset
import pandas as pd
import seaborn as sns
import numpy as np

In [ ]:
df = pd.read_csv('../../dataset/order_classification/edible_mushrooms.csv')

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
fig, ax = plt.subplots(figsize=(6, 4))
ax = sns.countplot(data = df, x = 'order', alpha = 0.7)

plt.setp(ax.get_xticklabels(), rotation=30, ha="right")
ax.set_xlabel('order')
ax.set_ylabel('nombre de photos')
ax.set_title('nombre de photos par species pour les champignons comestibles');

In [ ]:
max_size = df['order'].value_counts().max()
max_size = min(max_size, 5000)

In [ ]:
for index, value in df['order'].value_counts().items():
    print(f"{index}: {value}")

# Generating and balancing data

In [ ]:
path='../../dataset/class_order'
train_path='../../dataset/order_classification/train'
test_path='../../dataset/order_classification/test'
validation_path='../../dataset/order_classification/validation' 

In [ ]:
from tensorflow.keras.applications.mobilenet import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
train_data_generator = ImageDataGenerator(
    preprocessing_function = preprocess_input,
    shear_range = 0.2, # random application of shearing
    zoom_range = 0.2,
    rotation_range = 45,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    vertical_flip = True,
    fill_mode = 'nearest')

In [ ]:
train_generator = train_data_generator.flow_from_directory(
    directory = train_path,
    class_mode = "sparse",
    target_size = (224,224), batch_size = 16)

In [ ]:
import os 

# For each class, generate new images until it has as many images as the dominant class
for order, size in df['order'].value_counts().items():
    class_dir = os.path.join(train_path, order)
    num_new_images = max_size - size
    print(class_dir)

    i = 0
    for batch in train_data_generator.flow_from_directory(
        directory = train_path,
        batch_size = 16, 
        classes = [order],
        class_mode = "sparse",
        target_size = (224,224),
        save_to_dir = class_dir, 
        save_prefix='aug', 
        save_format='jpg'):

        i += 1
        if i >= num_new_images:
            break  # otherwise the generator would loop indefinitely

In [ ]:
test_data_generator = ImageDataGenerator(
    preprocessing_function = preprocess_input,
    shear_range = 0.2, # random application of shearing
    zoom_range = 0.2,
    rotation_range = 45,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    vertical_flip = True,
    fill_mode = 'nearest')

In [ ]:
test_generator = test_data_generator.flow_from_directory(
    directory = test_path,
    class_mode = "sparse",
    target_size = (224,224), batch_size = 16)

In [ ]:
import os 

max_size = min(max_size, 500)

# For each class, generate new images until it has as many images as the dominant class
for order, size in df['order'].value_counts().items():
    class_dir = os.path.join(test_path, order)
    num_new_images = max_size - size
    print(class_dir)

    i = 0
    for batch in test_data_generator.flow_from_directory(
        directory = test_path,
        batch_size = 16, 
        classes = [order],
        class_mode = "sparse",
        target_size = (224,224),
        save_to_dir = class_dir, 
        save_prefix='aug', 
        save_format='jpg'):

        i += 1
        if i >= num_new_images:
            break  # otherwise the generator would loop indefinitely